In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from coskweness_cokurtosis import coskewness, cokurtosis

stocks = ["AAPL", "GOOGL", "MSFT", "AMZN", "META", "TSLA", "WMT", "CAT", "KO", "IBM", "INTC", "CSCO", "ORCL", 
          "QCOM", "NVDA", "ADBE", "PYPL", "CRM", "ACN", "TXN", "AVGO", "INTU", "AMAT", "MU"]
#stocks = ["AAPL", "TSLA", "WMT", "CAT", "KO"]

data = yf.download(stocks, start="2019-01-01", end="2021-01-01")
data = yf.download(stocks, start="2019-04-23", end="2020-04-22")
#print(data.head())

# Compute expected returns in percentage
returns = data["Close"].pct_change().dropna()


mean_returns = returns.mean()
variance = returns.std()
skewness = returns.skew()
kurtosis = returns.kurtosis()
print("Mean returns: ", mean_returns*100)
print("Variance: ", variance*100)
print("Skewness: ", skewness)
print("Kurtosis: ", kurtosis)

mean_annual_returns = (1 + mean_returns)**252 - 1
variance_annual = variance*np.sqrt(252)

#print("Mean annual returns: ", mean_annual_returns*100)
#print("Variance annual: ", variance_annual*100)

[*********************100%***********************]  24 of 24 completed
[*********************100%***********************]  24 of 24 completed


Mean returns:  Ticker
AAPL     0.140660
ACN     -0.001056
ADBE     0.103107
AMAT     0.108966
AMZN     0.094221
AVGO    -0.025959
CAT     -0.057976
CRM      0.006215
CSCO    -0.088334
GOOGL    0.007062
IBM     -0.028195
INTC     0.040640
INTU     0.016499
KO       0.007699
META     0.000368
MSFT     0.152510
MU       0.052475
NVDA     0.200344
ORCL     0.011171
PYPL     0.040967
QCOM    -0.017235
TSLA     0.494667
TXN      0.011737
WMT      0.113878
dtype: float64
Variance:  Ticker
AAPL     2.566514
ACN      2.323285
ADBE     2.719120
AMAT     3.537940
AMZN     1.914542
AVGO     3.206679
CAT      2.508572
CRM      2.572148
CSCO     2.570782
GOOGL    2.270285
IBM      2.329367
INTC     3.109441
INTU     2.815855
KO       1.962097
META     2.418761
MSFT     2.501404
MU       3.599699
NVDA     3.493360
ORCL     2.542598
PYPL     2.783783
QCOM     2.980953
TSLA     4.760833
TXN      2.653173
WMT      1.814915
dtype: float64
Skewness:  Ticker
AAPL    -0.080396
ACN      0.730647
ADBE     0.4

In [2]:
#theta = 5
#utility_with_normality = mean_annual_returns - (1/2)*theta*variance_annual**2
#print("Utility with normality: ", utility_with_normality*100)
#utility_with_skewness_and_kurtosis = utility_with_normality + (1/6)*(theta**2)*(variance_annual**3)*skewness - (1/720)*(theta**3)*(variance_annual**4)*kurtosis
#print("Utility with skewness and kurtosis: ", utility_with_skewness_and_kurtosis*100)

In [3]:
from pypfopt import EfficientFrontier

risk_aversion = 5
ef = EfficientFrontier(mean_returns, returns.cov())
weights = ef.max_quadratic_utility(risk_aversion=risk_aversion)

print("Optimized Weights (considering variance and returns):")
for asset, weight in weights.items():
    print(f"{asset}: {weight:.2%}")

numpy_weights = np.array(list(weights.values()))

# Get utility with optimized weights
expected_returns = returns.mean()
expected_variance = returns.cov()
maximized_utility = -0.5 * risk_aversion * numpy_weights.T @ expected_variance @ numpy_weights + expected_returns @ numpy_weights

print("Maximized utility: ", maximized_utility)

Optimized Weights (considering variance and returns):
AAPL: 0.00%
ACN: 0.00%
ADBE: 0.00%
AMAT: 0.00%
AMZN: 0.00%
AVGO: 0.00%
CAT: 0.00%
CRM: 0.00%
CSCO: 0.00%
GOOGL: 0.00%
IBM: 0.00%
INTC: 0.00%
INTU: 0.00%
KO: 0.00%
META: 0.00%
MSFT: 0.00%
MU: 0.00%
NVDA: 0.00%
ORCL: 0.00%
PYPL: 0.00%
QCOM: 0.00%
TSLA: 41.17%
TXN: 0.00%
WMT: 58.83%
Maximized utility:  0.001308873286133041


In [4]:
import numpy as np
from scipy.optimize import minimize

numpy_returns = returns.to_numpy()

def standardize_higher_moments(moment):
    mean_c = np.mean(moment)
    std_c = np.std(moment)
    return (moment - mean_c) / std_c if std_c != 0 else moment

expected_returns = numpy_returns.mean(axis=0)
covariance_matrix = np.cov(numpy_returns, rowvar=False)
coskewness_matrix = coskewness(numpy_returns)
cokurtosis_matrix = cokurtosis(numpy_returns)

#for i in range(len(coskewness_matrix)):
#    print(coskewness_matrix[i][i][i], skewness.iloc[i])

#for i in range(len(cokurtosis_matrix)):
#    print(cokurtosis_matrix[i][i][i][i], kurtosis.iloc[i])

def objective(w):
    mu = np.dot(w, expected_returns)
    variance_term = (risk_aversion/2) * np.einsum('ij,i,j->', covariance_matrix, w, w)
    skewness_term = (risk_aversion**2/6) * np.einsum("ijk,i,j,k->", coskewness_matrix, w, w, w)
    kurtosis_term = (risk_aversion**3/24) * np.einsum("ijkl,k,j,i,l->", cokurtosis_matrix, w, w, w, w)
    value = -(mu - variance_term + skewness_term - kurtosis_term)
    return value

s = len(stocks)
constraint = lambda w: np.sum(w) - 1
w0 = np.ones(s)/s
bounds = [(0, 1)] * s
constraints = {'type': 'eq', 'fun': constraint}  # Equality constraint: sum(w) = 1

result = minimize(objective, w0, bounds=bounds, constraints=constraints, method = "COBYQA")

if result.success:
    weights = result.x
    print("Optimized weights:")
    for asset, weight in zip(mean_returns.index, weights):
        print(f"{asset}: {weight:.2%}")
else:
    print("Optimization failed:", result.message)


Optimized weights:
AAPL: 0.00%
ACN: 0.00%
ADBE: 0.00%
AMAT: 0.00%
AMZN: 8.90%
AVGO: 0.00%
CAT: 20.91%
CRM: 0.00%
CSCO: 0.00%
GOOGL: 0.00%
IBM: 0.00%
INTC: 0.00%
INTU: 0.00%
KO: 0.00%
META: 0.00%
MSFT: 0.00%
MU: 0.00%
NVDA: 0.00%
ORCL: 0.00%
PYPL: 0.00%
QCOM: 0.00%
TSLA: 45.43%
TXN: 0.00%
WMT: 24.77%
